# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис.  
Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах.  
То есть клиенты предлагают свои правки и комментируют изменения других.  
Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные.  
В вашем распоряжении набор данных с разметкой о токсичности правок.

**Постройте модель со значением метрики качества *F1* не меньше 0.75.**

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`.  
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import re

import nltk
import pandas as pd
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from tqdm.notebook import tqdm

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()

[nltk_data] Downloading package punkt to C:\Users\Александр
[nltk_data]     Куклин\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Александр
[nltk_data]     Куклин\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Александр
[nltk_data]     Куклин\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Загрузка и первичный анализ

In [3]:
data = pd.read_csv('toxic_comments.csv')

In [4]:
data.head()['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
data.duplicated().sum()

0

In [7]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

### Лемматизация и очистка текста

Лучший аналог учебного `pymystem3` — `WordNetLemmatizer` из `nltk` (подсказка из `Slack`).

In [8]:
def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.split())

def lemmatize(text):
    m = WordNetLemmatizer()
    txt_list =  nltk.word_tokenize(text)
    return ' '.join([m.lemmatize(word) for word in txt_list])

In [9]:
data['lemm_text'] = data['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [10]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not trying to edit war It ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### Подготовка выборок

In [11]:
train, test = train_test_split(data, test_size=0.1, random_state=12345)

target_train = train['toxic']
features_train = train['lemm_text']

target_test = test['toxic']
features_test = test['lemm_text']

print(
    f'Выборка разделена в пропорциях:\n'
    f'{(len(features_train)/len(data)):.0%} — тренировочная;\n'
    f'{(len(features_test)/len(data)):.0%} — тестовая;\n'
)

Выборка разделена в пропорциях:
90% — тренировочная;
10% — тестовая;



In [12]:
stopwords_ = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords_)
tf_idf = count_tf_idf.fit_transform(tqdm(features_train))

  0%|          | 0/143613 [00:00<?, ?it/s]

In [13]:
print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (143613, 154645)


In [14]:
tf_idf_test = count_tf_idf.transform(features_test)
print("Размер матрицы tf-idf_test:", tf_idf_test.shape)

Размер матрицы tf-idf_test: (15958, 154645)


***
### Вывод
- В комментариях нет дубликатов.
- В текстах комментариев встречаются «мусорные» символы, которые нужно убрать.
- Данные не сбалансированы по целевому признаку `toxic`.
- Провёл лемматизацию и очистил текст от лишних символов.
- Подготовил выборки. <br><br>
***

## Обучение

### Модель `Логистическя регрессия`

In [15]:
model = LogisticRegression(solver='liblinear', class_weight = 'balanced', random_state=12345)
parameters = {
    'C': [10],  # были варианты: 1, 5, 15
    'penalty': ['l2'],  # был вариант: 'l1'
    'max_iter': [100],  # были варианты: 50, 200, 250
}
lin_grid = GridSearchCV(model, parameters, scoring='f1', cv=3)

In [16]:
lin_grid.fit(tf_idf, target_train)
lin_fit_score = lin_grid.best_score_
print(f'Лучший показатель F1: {lin_fit_score:.3f}')

Лучший показатель F1: 0.763


### Модель `CatBoostClassifier`
Пытался подобрать параметры для модели методом `grid_search`,  
но подбор занимает крайне много времени, выставил параметры вручную, примерно.  

Попробую протестировать модель и посмотреть на оценку.  

In [17]:
cat_model = CatBoostClassifier(random_state=12345, verbose=False, eval_metric='F1', depth=6, learning_rate=0.5, iterations=200)
cat_model.fit(tf_idf, target_train)

### Модель `LGBMClassifier`

In [18]:
model = LGBMClassifier(random_state=12345, class_weight='balanced')
parameters = {
    'max_depth': [-1],  # были варианты: 5, 8, 10
    'num_leaves': [50],  # были варианты: 10, 50, 100
    'learning_rate': [0.1],  # были варианты: 0.1, 0.5, 1
    'n_estimators' : [1000],  # были варианты: 250, 650
}
lgbm_grid = GridSearchCV(model, parameters, scoring='f1', cv=3, n_jobs=-1)

In [19]:
lgbm_grid.fit(tf_idf, target_train)

GridSearchCV(cv=3,
             estimator=LGBMClassifier(class_weight='balanced',
                                      random_state=12345),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [-1],
                         'n_estimators': [1000], 'num_leaves': [50]},
             scoring='f1')

In [20]:
lgbm_fit_score = lgbm_grid.best_score_
print(
    f'Лучший показатель F1: {lgbm_fit_score:.3f}'
)

Лучший показатель F1: 0.768


***
### Вывод
- Обучил три модели.
- По модели `Логистическя регрессия` и `LGBMClassifier` удалось получить предварительный расчёт показателя `F1`,  
    по модели `CatBoostClassifier` не получилось из-за очень долгого времени обработки. <br><br>
***

## Тестирование
**Постройте модель со значением метрики качества F1 не меньше 0.75.**

### Модель `Логистическя регрессия`

In [21]:
predictions = lin_grid.predict(tf_idf_test)
lin_f1 = f1_score(target_test, predictions)
lin_conf_mat = confusion_matrix(target_test, predictions)

print(
    f'F1-score: {lin_f1:.3f}\n'
    f'Количество истинно отрицательных результатов: {lin_conf_mat[0][0]}\n'
    f'Количество ложноотрицательных результатов: {lin_conf_mat[1][0]}\n'
    f'Количество истинно положительных результатов: {lin_conf_mat[1][1]}\n'
    f'Количество ложноположительных результатов: {lin_conf_mat[0][1]}\n'
)

F1-score: 0.761
Количество истинно отрицательных результатов: 13732
Количество ложноотрицательных результатов: 298
Количество истинно положительных результатов: 1368
Количество ложноположительных результатов: 560



### Модель `CatBoostClassifier`

In [22]:
predictions = cat_model.predict(tf_idf_test)
cat_f1 = f1_score(target_test, predictions)
cat_conf_mat = confusion_matrix(target_test, predictions)

print(
    f'F1-score: {cat_f1:.3f}\n'
    f'Количество истинно отрицательных результатов: {cat_conf_mat[0][0]}\n'
    f'Количество ложноотрицательных результатов: {cat_conf_mat[1][0]}\n'
    f'Количество истинно положительных результатов: {cat_conf_mat[1][1]}\n'
    f'Количество ложноположительных результатов: {cat_conf_mat[0][1]}\n'
)

F1-score: 0.759
Количество истинно отрицательных результатов: 14156
Количество ложноотрицательных результатов: 565
Количество истинно положительных результатов: 1101
Количество ложноположительных результатов: 136



### Модель `LGBMClassifier`

In [23]:
predictions = lgbm_grid.predict(tf_idf_test)
lgbm_f1 = f1_score(target_test, predictions)
lgbm_conf_mat = confusion_matrix(target_test, predictions)

print(
    f'F1-score: {lgbm_f1:.3f}\n'
    f'Количество истинно отрицательных результатов: {lgbm_conf_mat[0][0]}\n'
    f'Количество ложноотрицательных результатов: {lgbm_conf_mat[1][0]}\n'
    f'Количество истинно положительных результатов: {lgbm_conf_mat[1][1]}\n'
    f'Количество ложноположительных результатов: {lgbm_conf_mat[0][1]}\n'
)

F1-score: 0.760
Количество истинно отрицательных результатов: 13855
Количество ложноотрицательных результатов: 377
Количество истинно положительных результатов: 1289
Количество ложноположительных результатов: 437



***
## Общий вывод
- Лучший результат показала модель `Логистическя регрессия`.  
    Достигнут целевой порог — F1-score выше 0.75.  
    Плюс у этой модели лучшай скорость обучения и предсказания.  
    Также у этой модели наименьший показатель ложноотрицательных результатов.  
    Для текущей задачи — отправки токсичных комментариев на модерацию — это наиболее существенный показатель.  
    Так меньше всего токсичных комментариев по ошибке минует модерацию. <br><br>
***